In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import re

In [2]:
#helper
def remove_numbers(string):
    """
    Remove some numbers and dots from a string for the sake of uniformity
    """
    step1 = re.sub("[0-9]", "", string)
    step2 = re.sub("\.", "", step1)
    
    return step2

In [3]:
# metadata
def find_metadata(piece_of_info):
    "Find the metadata given a dl element of class dl-horizontal"
    variable_name = []
    value = []
    
    if len(piece_of_info.find_all('dt')) == len(piece_of_info.find_all('dd')):
        for n, i in enumerate(piece_of_info.find_all('dt')):
            variable_name.append(i.text.strip())
            value.append(piece_of_info.find_all('dd')[n].text.strip())
    else:
        print("Error")
    
    return dict(zip(variable_name, value))

In [4]:
#then, the rest
def extract_sections(list_of_all_divs):
    """
    Create a clean dictionary from a soup.find_all("div") containing sections object. 
    For each section, create clean tex with the name of the section as key, and the text as value
    
    Input is something like:
    soup.find_all("div", {"class": re.compile("section*")})
    """
    sections = dict()
    
    for section in list_of_all_divs:
        
        try:
            title = remove_numbers(section.find("h2").text).strip()
            unwanted = section.find("h2")
            unwanted.extract()
        
            sections[title] = re.sub("\n", " ", section.text.strip())
        except:
            next
        
    return sections
    

In [5]:
# start with a list of search results
url = 'https://uitspraken.rechtspraak.nl/#zoekverfijn/so=Relevance&ps[]=ps1&cs[]=cs1&rg[]=r2&ins[]=itRechtbank&du[]=ud3&du[]=ud4'
#url = 'https://uitspraken.rechtspraak.nl/#zoekverfijn/dur[dr]=na&dur[da]=30-09-2022&dur[db]=&so=Relevance&ps[]=ps1&cs[]=cs1&rg[]=r2&ins[]=itRechtbank'

In [9]:
# retrieve all results 
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

for i in range(10000):
    time.sleep(0.2)
    try:
        elem = driver.find_element(By.XPATH, '//button[text()="Laad meer resultaten"]')
        elem.click()
    except: 
        break



In [10]:
# retrieve the html code of the entire expanded page
page = driver.execute_script('return document.body.innerHTML')
soup = BeautifulSoup(''.join(page), 'html.parser')


In [11]:
# extract links and metadata
stuff = soup.find_all("a", {"class": "js-titel titel"})
info = soup.find_all("dl", {"class": "dl-horizontal"})
links = []
metadata = {}
for n,i in enumerate(stuff):
    links.append(i['href'])
    metadata[i['href']] = find_metadata(info[n])

In [12]:
pd.DataFrame.from_dict(metadata, orient='index').reset_index().head(5)

,index,Datum uitspraak:,Datum publicatie:,Rechtsgebieden:,Bijzondere kenmerken:,Vindplaatsen:,Inhoudsindicatie:,Formele relaties:
0,https://uitspraken.rechtspraak.nl/inziendocume...,18-11-2022,18-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,NRC hoeft een artikel over twee fiscaal/juridi...,NaN
1,https://uitspraken.rechtspraak.nl/inziendocume...,17-11-2022,18-11-2022,Personen- en familierecht,"Eerste aanleg - enkelvoudig, Beschikking",Rechtspraak.nl,Geschil betreffende de (on)bevoegdheid Spaanse...,NaN
2,https://uitspraken.rechtspraak.nl/inziendocume...,17-11-2022,18-11-2022,Personen- en familierecht,"Eerste aanleg - enkelvoudig, Beschikking",Rechtspraak.nl,"Afwijzing verzoek vader tot gezamenlijk gezag,...",NaN
3,https://uitspraken.rechtspraak.nl/inziendocume...,17-11-2022,21-11-2022,Arbeidsrecht,Beschikking,Rechtspraak.nl,Rechtsgeldig ontslag op staande voet van PI-me...,NaN
4,https://uitspraken.rechtspraak.nl/inziendocume...,16-11-2022,16-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,Vordering tot ontruiming van krakers afgewezen...,NaN


In [13]:
def extract_data(links):
    """
    Input: a list of links
    Extract detailed court data on the basis of a list of links
    The function goes through various steps:
    For each page: 
    (1): Direct Selenium towards the appropriate page and parse the html
    (2): Download the direct metadata stored in dl-horizontal
    (3): Download the central information
    (4): Extract all the other sections and their text
    (5): Return a dictionary with all of those things
    """
    output = dict()
    
    for i in links:
        
        rechtspraak_data = dict()
        driver.get(i)
        page = driver.execute_script('return document.body.innerHTML')
        soup = BeautifulSoup(''.join(page), 'html.parser')
    
        ## first, metadata inside document
        meta_inside = soup.find("dl", {"class": "dl-horizontal"})
        metadata_inside = find_metadata(meta_inside)
    
        rechtspraak_data['metadata_inside'] = metadata_inside
    
        #then, uitspraak info
        try:
            uitspraak_info = soup.find_all("div", {"class": "uitspraak-info"})
            uitspraak_info_neat = [i.text.strip() for i in uitspraak_info[0].find_all("div")]
            rechtspraak_data['uitspraak_info'] = uitspraak_info_neat
            
        except:
            rechtspraak_data['uitspraak_info'] = "NA"
    
        allothersections = soup.find_all("div", {"class": re.compile("section*")})
        rechtspraak_data['sections'] = extract_sections(allothersections)
    
        output[i] = rechtspraak_data
        
    
    return output

In [16]:
test = extract_data(links)

In [20]:
test2 = pd.DataFrame.from_dict(test, orient='index').reset_index()
test3 = pd.DataFrame.from_dict(test2['sections'])['sections'].apply(pd.Series)


In [23]:
# this is metadata from outside the court cases
pd.DataFrame.from_dict(metadata, orient='index').reset_index().head(5)


,index,Datum uitspraak:,Datum publicatie:,Rechtsgebieden:,Bijzondere kenmerken:,Vindplaatsen:,Inhoudsindicatie:,Formele relaties:
0,https://uitspraken.rechtspraak.nl/inziendocume...,18-11-2022,18-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,NRC hoeft een artikel over twee fiscaal/juridi...,NaN
1,https://uitspraken.rechtspraak.nl/inziendocume...,17-11-2022,18-11-2022,Personen- en familierecht,"Eerste aanleg - enkelvoudig, Beschikking",Rechtspraak.nl,Geschil betreffende de (on)bevoegdheid Spaanse...,NaN
2,https://uitspraken.rechtspraak.nl/inziendocume...,17-11-2022,18-11-2022,Personen- en familierecht,"Eerste aanleg - enkelvoudig, Beschikking",Rechtspraak.nl,"Afwijzing verzoek vader tot gezamenlijk gezag,...",NaN
3,https://uitspraken.rechtspraak.nl/inziendocume...,17-11-2022,21-11-2022,Arbeidsrecht,Beschikking,Rechtspraak.nl,Rechtsgeldig ontslag op staande voet van PI-me...,NaN
4,https://uitspraken.rechtspraak.nl/inziendocume...,16-11-2022,16-11-2022,Civiel recht,Kort geding,Rechtspraak.nl,Vordering tot ontruiming van krakers afgewezen...,NaN


In [24]:
# this is metadata from within the court cases
pd.DataFrame.from_dict(test2['metadata_inside'])['metadata_inside'].apply(pd.Series).head(5)

,Instantie,Datum uitspraak,Datum publicatie,Zaaknummer,Rechtsgebieden,Bijzondere kenmerken,Inhoudsindicatie,Vindplaatsen,,Formele relaties,Wetsverwijzingen
0,Rechtbank Amsterdam,18-11-2022,18-11-2022,C/13/724069 / KG ZA 22-894,Civiel recht,Kort geding,NRC hoeft een artikel over twee fiscaal/juridi...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
1,Rechtbank Gelderland,17-11-2022,18-11-2022,C/05/406723 / FA RK 22-2259,Personen- en familierecht,Eerste aanleg - enkelvoudig\nBeschikking,Geschil betreffende de (on)bevoegdheid Spaanse...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
2,Rechtbank Gelderland,17-11-2022,18-11-2022,C/05/403829 / FA RK 22-1457,Personen- en familierecht,Eerste aanleg - enkelvoudig\nBeschikking,"Afwijzing verzoek vader tot gezamenlijk gezag,...",Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
3,Rechtbank Rotterdam,17-11-2022,21-11-2022,10081293,Arbeidsrecht,Beschikking,Rechtsgeldig ontslag op staande voet van PI-me...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN
4,Rechtbank Amsterdam,16-11-2022,16-11-2022,C/13/724873 / KG ZA 22-930,Civiel recht,Kort geding,Vordering tot ontruiming van krakers afgewezen...,Rechtspraak.nl,Verrijkte uitspraak,NaN,NaN


In [25]:
#export both to csv
pd.DataFrame.from_dict(metadata, orient='index').reset_index().to_csv("metadata_out1.csv", sep = "\t")
pd.DataFrame.from_dict(test2['metadata_inside'])['metadata_inside'].apply(pd.Series).to_csv("metadata_in1.csv", sep ='\t')

In [22]:
#export the content to csv
#test3.iloc[:,0:7].to_csv("data1.csv", sep = "\t")
test3[['De procedure', 'Het verloop van de procedure', 'Het geschil', 'De beoordeling', 'De feiten', 'Beslissing',  'De beslissing']].to_csv("data1.csv", sep = "\t")

In [ ]:
driver.close()